# Data Cleaning

On this notebook we created a dataframe with the results of the FPT for each individual and for 50 different circles radii. The radii range is from 12 to 250 meters, the lower due to the GPS error and the higher because it is a quarter of the net displacement of the species (see Barraquand & Benhamou, 2008).


## Input
  **file_1**: FPT results for 50 radii of each individual and trajectory , obtained using R adehabitatLT package. The file name correspond to the individual and it's trajectory. For example: the 2nd trajectory of the 1st indivivual is "ID1.2".
  
  **file_2**: different trajectories attributes as the coordinates, time lag, date, etc. The file name correspond to the individual and it's trajectory. For example: the 2nd trajectory of the 1st indivivual is "ID_1.2".
  
  **idx**: dataframe index. The file name correspond to the individual and it's trajectory. For example: the 2nd trajectory of the 1st indivivual is "idx1.2".

## Output

**new_bigdf** : data frame with the woodpecker ID, the date, the coordinates, the relocations and the FPT for the 50 radii.

## References

BARRAQUAND F & S BENHAMOU (2008) Animal movements in heterogeneous landscapes: Identifying profitable places and homogeneous movement bouts. Ecology 89(12): 3336-3348.







In [1]:
import pandas as pd
from rpy2.robjects import r
import rpy2.robjects.pandas2ri as pandas2ri


We created a **"read"** function, which open the R file and join the different dataframes of the FPT results with the coordinates of the individuals. There were results just for 18 individuals, due to the exclusion of 6 irregular trajectories.


In [2]:
def read (file_1, file_2, idx,x,y):
    from pathlib import Path
    my_file= Path(file_1)

    if my_file.is_file(): 
        #Load R file
        rf=r['load'](file_1) 
        df=pandas2ri.ri2py_dataframe(r[rf[0]])

        #Transform the R file to a datsaframe 
        datadf =[]
        for col in df.columns:
            datadf.append(pd.Series(list(df.iloc[0][col])))
        newdf = pd.DataFrame(data = datadf).T

        #Add new columns
        reloc=pd.read_csv(idx,header=None, names=None)
        newdf.columns= ["FPT" + str (i) for i in range (1,51)]
        newdf.insert(0, "ID", x)
        newdf.insert(1, "Reloc",reloc[0])
        newdf.insert(2, "Day", y)

        #Load R file
        rf2=r['load'](file_2)
        df2=pandas2ri.ri2py_dataframe(r[rf2[0]])

        #Add columns
        df2.insert(0, "Reloc", df2.index)
        df2['Reloc'] = pd.to_numeric(df2['Reloc'])
        
        #Drop columns that are not necessary
        df2=df2.drop(['dx', 'dy','R2n', 'abs.angle', 'rel.angle'], axis="columns")
        
        #Change date format to datetime
        df2['date']=df2.date.map(lambda x: pd.to_datetime(x, unit='s'))

        #Join dataframe on the relocation column
        id_join=pd.merge(newdf, df2, on="Reloc", how="inner") 
        #Dataframe for one trajectory of one woodpecker
        
        return id_join
    
    else:
       
        return None


        


                

        
        
            
          
                
                
    

In [ ]:
PATH='No Interpolados/' #Files Path

In [3]:

id_todos= []
#A loop that applies the read function to every file
for x in range (1,19):
    for y in range (1,9):
        file_1= PATH +'ID'+ str(x) + '.' + str(y)
        file_2= PATH +'ID' + '_' + str(x) + '.' + str(y)
        idx= PATH +'idx' + str(x) + '.' + str(y) + '.' + 'csv'        
        df=read(file_1, file_2, idx, x, y) 
        if df is not None:
            #Creates a list of the FPT results and trajectory attributes of every individual
            id_todos.append(df) 
            
#Creates a dataframe of the FPT results and trajectory attributes of every individual        
bigdf= pd.concat(id_todos) 
    

        
        
        
       

In [4]:
bigdf

,ID,Reloc,Day,FPT1,FPT2,FPT3,FPT4,FPT5,FPT6,FPT7,...,FPT46,FPT47,FPT48,FPT49,FPT50,x,y,date,dist,dt
0,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,579778.3163,3912570.091,2014-10-08 18:02:00,14.882624,120.0
1,1,2,1,113.059505,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,579768.7550,3912558.686,2014-10-08 18:04:00,88.330710,120.0
2,1,3,1,35.961147,50.516849,65.072551,79.628253,94.183956,108.739658,123.295360,...,NaN,NaN,NaN,NaN,NaN,579800.3625,3912476.204,2014-10-08 18:06:00,73.249729,120.0
3,1,4,1,43.683414,61.364795,79.046177,96.727559,114.408940,132.090322,149.771703,...,NaN,NaN,NaN,NaN,NaN,579785.6257,3912404.452,2014-10-08 18:08:00,59.938469,120.0
4,1,5,1,42.489283,59.687327,76.885370,94.083413,111.281457,128.479500,145.677543,...,NaN,NaN,NaN,NaN,NaN,579777.6466,3912345.047,2014-10-08 18:10:00,77.986874,120.0
5,1,6,1,69.303879,97.355449,125.407020,153.458590,277.420294,455.552630,552.293270,...,5324.927310,5353.968160,5383.009009,5412.049859,5441.090708,579741.8035,3912275.785,2014-10-08 18:12:00,28.324581,120.0
6,1,7,1,152.519163,214.253110,275.987057,337.721004,401.257139,465.811160,530.365181,...,5476.050232,5504.051143,5532.052053,5560.052964,5588.053875,579763.7852,3912293.648,2014-10-08 18:14:00,NaN,120.0
7,1,10,1,124.077987,174.300030,224.522072,274.744115,324.966157,375.188200,425.410242,...,5406.613674,5430.623917,5454.634160,5478.644403,5502.654646,579724.7281,3912310.370,2014-10-08 18:20:00,64.291290,120.0
8,1,11,1,51.421982,72.235641,93.049301,113.862960,134.676619,155.490279,176.303938,...,9669.651089,NaN,NaN,NaN,NaN,579784.0101,3912335.250,2014-10-08 18:22:00,49.614242,120.0
9,1,12,1,170.590982,223.892144,277.193306,735.471011,778.414284,996.989863,1059.720737,...,5451.989707,5476.891018,5501.792330,5526.693641,5551.594953,579743.7552,3912306.248,2014-10-08 18:24:00,9.479043,120.0


In [5]:
#Tidy dataframe
tidy_bigdf= pd.melt(bigdf, ["ID","date", "Day", "Reloc", "x", "y", "dt",'dist'], var_name="FPT_n", value_name="FPT_value")

In [6]:
tidy_bigdf.head()

,ID,date,Day,Reloc,x,y,dt,dist,FPT_n,FPT_value
0,1,2014-10-08 18:02:00,1,1,579778.3163,3912570.091,120.0,14.882624,FPT1,NaN
1,1,2014-10-08 18:04:00,1,2,579768.7550,3912558.686,120.0,88.330710,FPT1,113.059505
2,1,2014-10-08 18:06:00,1,3,579800.3625,3912476.204,120.0,73.249729,FPT1,35.961147
3,1,2014-10-08 18:08:00,1,4,579785.6257,3912404.452,120.0,59.938469,FPT1,43.683414
4,1,2014-10-08 18:10:00,1,5,579777.6466,3912345.047,120.0,77.986874,FPT1,42.489283


Check if all the date is in the dataframe.
It is necessary to divide the number of data by 50, because every trajectory has the result of FPT for 50 radii. This result should be the number of data of just one trajectory (Day) for each individual.

In [7]:
check=tidy_bigdf.groupby(["ID","Day"]).size()/50
check

ID  Day
1   1      125.0
    2      255.0
    3      289.0
    4      311.0
    5       30.0
2   1       38.0
    2      100.0
    3       92.0
    4       87.0
3   1       36.0
    2       71.0
    3       62.0
    4       47.0
    5       71.0
    6       64.0
    7       62.0
    8        3.0
4   1       41.0
    2       68.0
    3       59.0
    4       72.0
    5       36.0
    6        6.0
5   1       21.0
    2       52.0
6   1       11.0
    2       86.0
7   1       47.0
8   1       63.0
    2      120.0
           ...  
9   2       68.0
    3       56.0
    4       55.0
    5       55.0
10  1        6.0
    2       38.0
    3       32.0
    4       47.0
    5       40.0
11  1       57.0
    2      129.0
    3       48.0
12  1       33.0
    2       32.0
    3       16.0
13  1       21.0
14  1       12.0
    2       60.0
    3       15.0
15  1       15.0
    2        5.0
    3        4.0
16  1       14.0
    2       16.0
17  1       10.0
    2      109.0
18  1       85.0
    2 

Add the values of the radii to the dataframe

In [8]:
radio= pd.read_csv("radio.csv", names=['radio']) 
#The file radio.csv contains the values of the radii from 12 to 250 (m).

In [9]:
#The FPT_n are related to the index of the radio.csv file 
radio = radio.assign(FPT_n = ['FPT{}'.format(1+i) for i in range(50)]) 
radio.head()

,radio,FPT_n
0,12.00000,FPT1
1,16.85714,FPT2
2,21.71429,FPT3
3,26.57143,FPT4
4,31.42857,FPT5


In [10]:
#Dataframe with all the trajectories data and the radii values
new_bigdf=pd.merge(tidy_bigdf, radio, on="FPT_n", how="inner")

In [11]:
new_bigdf.head()

,ID,date,Day,Reloc,x,y,dt,dist,FPT_n,FPT_value,radio
0,1,2014-10-08 18:02:00,1,1,579778.3163,3912570.091,120.0,14.882624,FPT1,NaN,12.0
1,1,2014-10-08 18:04:00,1,2,579768.7550,3912558.686,120.0,88.330710,FPT1,113.059505,12.0
2,1,2014-10-08 18:06:00,1,3,579800.3625,3912476.204,120.0,73.249729,FPT1,35.961147,12.0
3,1,2014-10-08 18:08:00,1,4,579785.6257,3912404.452,120.0,59.938469,FPT1,43.683414,12.0
4,1,2014-10-08 18:10:00,1,5,579777.6466,3912345.047,120.0,77.986874,FPT1,42.489283,12.0


In [12]:
#Save the file for posterior analysis (go to Variance notebook)
new_bigdf.to_csv("newbigdf_clean.csv", index=False) 